In [89]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.lotsawahouse.org/topics/"

page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
series = soup.findAll('a', {'class':'index-entry'})

for serie in series:
    URL = 'https://www.lotsawahouse.org/'+serie['href']
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    # get titles from html
    titles = soup.findAll('a', {'class':'title'})

    for title in titles:
        try:
            # get html for title
            URL = 'https://www.lotsawahouse.org/' + title['href']
            page = requests.get(URL)

            soup = BeautifulSoup(page.content, "html.parser")

            # extract text from html
            maintext = soup.find('div', {'id': "maintext"})
            tib = maintext.findAll('p', {'class': 'TibetanVerse'})
            phon = maintext.findAll('p', {'class': 'EnglishPhonetics'})

            if len(tib) == len(phon): # if pairs are valid, save them

                # prep pairs from text
                tib = [elt.contents[0].replace('\xa0', '') for elt in tib]
                phon = [elt.contents[0].replace(',', '') for elt in phon]
                pairs = [(tib_elt + ',' + phon_elt + '\n') for tib_elt, phon_elt in zip(tib, phon)]

                # write pairs to csv
                with open('pairs.csv', 'a') as f:
                    for pair in pairs:
                        f.write(pair)
        except:
            pass